# Case When Lab

### Introduction

In this lesson, we'll use case when to work with data of different purchases.

### Loading our Data

For this lab, we'll udate from DBT's sample database.

In [1]:
import sqlite3
conn = sqlite3.connect('furniture_shop.db')

In [2]:
import pandas as pd
root_url = "https://raw.githubusercontent.com/data-eng-10-21/case-when/main/data/"
orders_df = pd.read_csv(f'{root_url}/updated_orders.csv', parse_dates=['order_date'])
payments_df = pd.read_csv(f'{root_url}/payments.csv')
customers_df = pd.read_csv(f'{root_url}/customers.csv')

In [3]:
orders_df.to_sql('orders', conn, index = False, if_exists = 'replace')
payments_df.to_sql('payments', conn, index = False, if_exists = 'replace')
customers_df.to_sql('customers', conn, index = False, if_exists = 'replace')

### Exploring our data

In [48]:
query = """
SELECT * FROM customers LIMIT 3;
"""
pd.read_sql(query, conn)

,id,first_name,last_name
0,1,Michael,P.
1,2,Shawn,M.
2,3,Kathleen,P.


Now let's begin by working with our order data.

In [4]:
query = """
SELECT * FROM orders LIMIT 3;
"""
pd.read_sql(query, conn)

,id,user_id,order_date,status
0,1,1,2018-01-01 00:00:00,returned
1,2,3,2018-01-02 00:00:00,completed
2,3,94,2018-01-04 00:00:00,completed


We can see that each order has a different status.  Let's begin by viewing all of the different options under status.  Name of the column `status_types`.

In [5]:
query = """
SELECT DISTINCT(status) as status_types
FROM orders
"""
pd.read_sql(query, conn)

# 	status_types
# 0	returned
# 1	completed
# 2	return_pending
# 3	shipped
# 4	placed

,status_types
0,returned
1,completed
2,return_pending
3,shipped
4,placed


And now let's count up the number of times each status occurs in our dataset, order from most to least. 

In [21]:
query = """
SELECT status,
COUNT(CASE WHEN status = 'returned' THEN 1 END) as amount,
COUNT(CASE WHEN status = 'completed' THEN 1 END) as amount,
COUNT(CASE WHEN status = 'return_pending' THEN 1 END) as amount,
COUNT(CASE WHEN status = 'shipped' THEN 1 END) as amount,
COUNT(CASE WHEN status = 'placed' THEN 1 END) as amount
FROM orders
GROUP BY status
ORDER BY amount DESC
"""
pd.read_sql(query, conn)

# 	status	amount
# 0	completed	50
# 1	returned	33
# 2	shipped	7
# 3	placed	7
# 4	return_pending	2

,status,amount,amount,amount,amount,amount
0,returned,33,0,0,0,0
1,completed,0,50,0,0,0
2,placed,0,0,0,0,7
3,return_pending,0,0,2,0,0
4,shipped,0,0,0,7,0


Now let's try to get information on orders of each customer.  For each customer, count up the number of times he occurs in the orders database, as well as the number of orders that were completed and number of orders that were returned.  Order the data by the number of orders, and display the top five customers.

In [27]:
query = """
SELECT user_id, count(*) as total_orders,
COUNT(CASE WHEN status='returned' THEN 1 END) as returned,
COUNT(CASE WHEN status='completed' THEN 1 END) as completed
FROM orders
GROUP BY user_id
ORDER BY total_orders DESC
LIMIT 5
"""
pd.read_sql(query, conn)

# 	user_id	total_orders	returned	completed
# 0	64	2	2	0
# 1	51	3	2	1
# 2	46	2	2	0
# 3	27	2	2	0
# 4	99	2	1	1

,user_id,total_orders,returned,completed
0,54,5,0,3
1,3,3,1,2
2,22,3,0,2
3,51,3,2,1
4,66,3,1,1


### Working with Payments

Next, let's work with our payments data.

In [28]:
query = """
SELECT * FROM payments
LIMIT 1
"""
pd.read_sql(query, conn)

,id,order_id,payment_method,amount
0,1,1,credit_card,1000


Let's begin by viewing the amount of times each payment method was used, and order from most to least.

In [41]:
query = """
SELECT payment_method,
COUNT(CASE WHEN payment_method = 'credit_card' THEN 1 END) as amount,
COUNT(CASE WHEN payment_method = 'bank_transfer' THEN 1 END) as amount,
COUNT(CASE WHEN payment_method = 'coupon' THEN 1 END) as amount,
COUNT(CASE WHEN payment_method = 'gift_card' THEN 1 END) as amount
FROM payments
GROUP BY payment_method
ORDER BY amount DESC
"""
pd.read_sql(query, conn)

# 	payment_method	amount
# 0	credit_card	55
# 1	bank_transfer	33
# 2	coupon	13
# 3	gift_card	12

,payment_method,amount,amount,amount,amount
0,credit_card,55,0,0,0
1,bank_transfer,0,33,0,0
2,coupon,0,0,13,0
3,gift_card,0,0,0,12


Now for each customer, calculate the amount spent via credit card, bank transfer, other (if not spent by credit card or bank transfer), and the total amount spent.  Order by the total amount spent.  Limit to the first five customers.

In [54]:
query = """
SELECT user_id, SUM(amount) as total_spend,
SUM(CASE WHEN payment_method = 'credit_card' THEN amount ELSE 0 END) as credit_spend,
SUM(CASE WHEN payment_method = 'bank_transfer' THEN amount ELSE 0 END) as bank_spend,
SUM(CASE WHEN payment_method NOT IN ('credit_card', 'bank_transfer') THEN amount ELSE 0 END) as other_spend
FROM payments p
JOIN orders o
ON p.order_id = o.id
GROUP BY user_id
ORDER BY total_spend DESC
LIMIT 5
"""
pd.read_sql(query, conn)

# 	user_id	total_spend	credit_spend	bank_spend	other_spend
# 0	51	9900	6300	2000	1600
# 1	3	6500	3900	0	2600
# 2	46	6400	3400	0	3000
# 3	54	5700	2500	2600	600
# 4	30	5700	2900	0	2800

,user_id,total_spend,credit_spend,bank_spend,other_spend
0,51,9900,6300,2000,1600
1,3,6500,3900,0,2600
2,46,6400,3400,0,3000
3,30,5700,2900,0,2800
4,54,5700,2500,2600,600


Now the above query includes returned orders, so let's perform the same query but this time update the query to exclude returned items.

In [ ]:
query = """

"""
pd.read_sql(query, conn)

# user_id	total_spend	credit_spend	bank_spend	other_spend
# 0	51	5800	2200	2000	1600
# 1	54	5700	2500	2600	600
# 2	22	5200	0	500	4700
# 3	50	4700	2200	0	2500
# 4	71	4200	4200	0	0

,user_id,total_spend,credit_spend,bank_spend,other_spend
0,51,5800,2200,2000,1600
1,54,5700,2500,2600,600
2,22,5200,0,500,4700
3,50,4700,2200,0,2500
4,71,4200,4200,0,0


### Summary

In this lesson, we saw how we can use case when statement to perform calculations on various customer orders.  

### Resources

[Original data](https://github.com/dbt-labs/jaffle_shop)